In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import scipy.io as spio
data = {}
test_data = {}
# blocks 1 and 3 are gain and have 4 for win and 3 for loss
# blocks 2 and 4 are loss and have 2 for win and 1 for loss
for i in range(1,101):
    testData = spio.loadmat('genData_smG_rlG_stickyActG_stickyStimGL_wslsG_hG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    block = [
                {
                    'action': np.array([[resp-1 for ([id],[resp]) in zip(struct[0,0]['blockID'],struct[0,0]['selectStimIndex']) if id==blockID]]),
                    'state': np.array([[stimOffers for ([id],stimOffers) in zip(struct[0,0]['blockID'],struct[0,0]['stimOffers']) if id==blockID]]),
                    'reward': np.array([[id%2+int(rew/25)+2 for ([id],[rew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew']) if id==blockID]]),
                    'id': 'S'+str(i),

                    'block': blockID
                } for blockID in np.unique([struct[0,0]['blockID']])

            ]
#     block = [
#                 {
#                     'action': np.array([struct[0,0]['selectStimIndex'].flatten()])-1,
#                     'state': np.array([struct[0,0]['stimOffers']]),
#                     'reward': np.array([[id%2+int(rew/25)+2 for ([id],[rew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew'])]]),
#                     'id': 'S'+str(i),

#                     'block': 0
#                 }

#             ]
    if i<91:
        data['S'+str(i)] = block
    else:
        test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[2, 1, 2, 0, 2, 1, 2, 1, 2, 0, 2, 0, 1, 0, 1, 0, 1, 2, 1, 1, 2, 1,
           2, 2, 0, 1, 2, 0, 1, 1, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 1, 2, 2, 0,
           0, 1, 2, 2]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [3, 1],
           [2, 1],
           [1, 3],
           [2, 1],
           [2, 3],
           [3, 1],
           [3, 1],
           [1, 2],
           [2, 3],
           [1, 2],
           [1, 2],
           [1, 2],
           [2, 1],
           [3, 2],
           [3, 2],
           [2, 3],
           [3, 1],
           [2, 3],
           [1, 3],
           [3, 2],
           [2, 1],
           [2, 1],
           [3, 2],
           [1, 3],
           [2, 1],
           [2, 1],
           [2, 3],
           [1, 2],
           [2, 3],
           [3, 2],
           [1, 3],
           [3, 1],
           [3, 2],
           [3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [2, 3

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=3, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/fullModel_selectedStimulus_rewardOutcome/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-02 14:30:40,782 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:30:40,784 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:30:40,785 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:30:40,787 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:30:40,788 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-02 14:31:01,988 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/fullModel_selectedStimulus_rewardOutcome/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:31:45,085 - DeepRL - DEBUG - global iter =    0 total obj: 17453.7457
2021-07-02 14:32:13,468 - DeepRL - DEBUG - global iter =    1 total obj: 14392.2216
2021-07-02 14:32:40,950 - DeepRL - DEBUG - global iter =    2 total obj: 13287.9500
2021-07-02 14:33:08,199 - DeepRL - DEBUG - global iter =    3 total obj: 12968.0101
2021-07-02 14:33:36,286 - DeepRL - DEBUG - global iter =    4 total obj: 12820.4653
2021-07-02 14:34:04,876 - DeepRL - DEBUG - global iter =    5 total obj: 12727.3510
2021-07-02 14:34:33,285 - DeepRL - DEBUG - global iter =    6 total obj: 12652.3623
2021-07-02 14:35:01,807 - DeepRL - DEBUG - global iter =    7 total obj: 12581.7871
2021-07-02 14:35:29,897 - DeepRL - DEBUG - global iter =    8 total obj: 12515.4496
2021-07-02 14:35:54,594 - DeepRL - DEBUG - global iter =    9 total obj: 12410.4110
2021-07-02 14:36:19,280 - DeepRL - DEBUG - global iter =   10 total obj: 12284.4591
2021-07-02 14:36:44,381 - DeepRL - DEBUG - global iter =   11 total obj: 119

INFO:tensorflow:../results/fullModel_selectedStimulus_rewardOutcome/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[4, 3, 4, ..., 4, 4, 4],
          [2, 1, 2, ..., 2, 2, 2],
          [4, 3, 4, ..., 3, 3, 4],
          ...,
          [2, 1, 2, ..., 2, 2, 2],
          [4, 4, 4, ..., 3, 4, 3],
          [2, 2, 2, ..., 1, 2, 1]], dtype=int32),
   'action': array([[2, 1, 2, ..., 1, 2, 2],
          [2, 1, 2, ..., 1, 2, 2],
          [0, 0, 0, ..., 0, 0, 2],
          ...,
          [2, 1, 2, ..., 1, 2, 2],
          [0, 2, 0, ..., 0, 2, 0],
          [0, 2, 0, ..., 0, 2, 0]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[1, 2],
           [3, 1],
           [1, 3],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          ...,
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ..

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-02 14:50:27,522 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:50:27,524 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:50:27,525 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:50:27,527 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:50:27,528 - DeepRL - DEBUG - test data-points: 10
2021-07-02 14:50:40,920 - DeepRL - DEBUG - opt started...
2021-07-02 14:50:40,922 - DeepRL - DEBUG - started testing...
2021-07-02 14:52:11,007 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_selectedStimulus_rewardOutcome/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:52:41,884 - DeepRL - DEBUG - global iter =    0 total obj: 17555.5305
2021-07-02 14:52:57,820 - DeepRL - DEBUG - global iter =    1 total obj: 14909.4315
2021-07-02 14:53:15,065 - DeepRL - DEBUG - global iter =    2 total obj: 13253.0114
2021-07-02 14:53:32,172 - DeepRL - DEBUG - global iter =    3 total obj: 12898.9264
2021-07-02 14:53:48,024 - DeepRL - DEBUG - global iter =    4 total obj: 12727.8686
2021-07-02 14:54:06,859 - DeepRL - DEBUG - global iter =    5 total obj: 12622.4682
2021-07-02 14:54:24,087 - DeepRL - DEBUG - global iter =    6 total obj: 12548.2946
2021-07-02 14:54:41,552 - DeepRL - DEBUG - global iter =    7 total obj: 12491.5160
2021-07-02 14:54:58,798 - DeepRL - DEBUG - global iter =    8 total obj: 12441.7920
2021-07-02 14:55:16,242 - DeepRL - DEBUG - global iter =    9 total obj: 12398.6373
2021-07-02 14:55:16,243 - DeepRL - DEBUG - started testing...
2021-07-02 14:56:53,343 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_selectedStimulus_rewardOutcome/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:57:17,987 - DeepRL - DEBUG - global iter =   10 total obj: 12363.6847
2021-07-02 14:57:32,709 - DeepRL - DEBUG - global iter =   11 total obj: 12328.8409
2021-07-02 14:57:49,871 - DeepRL - DEBUG - global iter =   12 total obj: 12301.2900
2021-07-02 14:58:06,904 - DeepRL - DEBUG - global iter =   13 total obj: 12266.7164
2021-07-02 14:58:24,066 - DeepRL - DEBUG - global iter =   14 total obj: 12238.9462
2021-07-02 14:58:39,132 - DeepRL - DEBUG - global iter =   15 total obj: 12209.8974
2021-07-02 14:58:53,004 - DeepRL - DEBUG - global iter =   16 total obj: 12186.1899
2021-07-02 14:59:05,625 - DeepRL - DEBUG - global iter =   17 total obj: 12167.1341
2021-07-02 14:59:19,953 - DeepRL - DEBUG - global iter =   18 total obj: 12150.9117
2021-07-02 14:59:36,829 - DeepRL - DEBUG - global iter =   19 total obj: 12124.6104
2021-07-02 14:59:36,831 - DeepRL - DEBUG - started testing...
2021-07-02 15:01:45,179 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/fullModel_selectedStimulus_rewardOutcome/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:06:34,612 - DeepRL - DEBUG - global iter =   30 total obj: 11770.9063
2021-07-02 15:06:48,857 - DeepRL - DEBUG - global iter =   31 total obj: 11727.2434
2021-07-02 15:07:03,116 - DeepRL - DEBUG - global iter =   32 total obj: 11652.5107
2021-07-02 15:07:17,314 - DeepRL - DEBUG - global iter =   33 total obj: 11599.3824
2021-07-02 15:07:28,765 - DeepRL - DEBUG - global iter =   34 total obj: 11563.7757
2021-07-02 15:07:39,471 - DeepRL - DEBUG - global iter =   35 total obj: 11494.3799
2021-07-02 15:07:49,406 - DeepRL - DEBUG - global iter =   36 total obj: 11444.5912
2021-07-02 15:08:00,182 - DeepRL - DEBUG - global iter =   37 total obj: 11417.4256
2021-07-02 15:08:11,516 - DeepRL - DEBUG - global iter =   38 total obj: 11384.0446
2021-07-02 15:08:22,772 - DeepRL - DEBUG - global iter =   39 total obj: 11350.4954
2021-07-02 15:08:22,774 - DeepRL - DEBUG - started testing...
2021-07-02 15:10:49,000 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_selectedStimulus_rewardOutcome/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:11:19,671 - DeepRL - DEBUG - global iter =   40 total obj: 11287.1397
2021-07-02 15:11:32,531 - DeepRL - DEBUG - global iter =   41 total obj: 11230.5420
2021-07-02 15:11:43,311 - DeepRL - DEBUG - global iter =   42 total obj: 11210.4013
2021-07-02 15:11:52,610 - DeepRL - DEBUG - global iter =   43 total obj: 11182.8109
2021-07-02 15:12:01,645 - DeepRL - DEBUG - global iter =   44 total obj: 11184.7819
2021-07-02 15:12:09,673 - DeepRL - DEBUG - global iter =   45 total obj: 11157.0479
2021-07-02 15:12:16,795 - DeepRL - DEBUG - global iter =   46 total obj: 11152.2577
2021-07-02 15:12:22,914 - DeepRL - DEBUG - global iter =   47 total obj: 11100.4144
2021-07-02 15:12:28,749 - DeepRL - DEBUG - global iter =   48 total obj: 11131.4256
2021-07-02 15:12:34,591 - DeepRL - DEBUG - global iter =   49 total obj: 11063.6215
2021-07-02 15:12:34,592 - DeepRL - DEBUG - opt finished.
2021-07-02 15:12:34,593 - DeepRL - DEBUG - started testing...
2021-07-02 15:12:41,064 - DeepRL - DEBUG 

INFO:tensorflow:../results/fullModel_selectedStimulus_rewardOutcome/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[1, 2, 2, 1, 2, 2, 1, 1, 1, 0, 2, 1, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2,
          2, 2, 2, 1, 2, 0, 2, 0, 1, 1, 1, 2, 1, 2, 0, 0, 1, 0, 1, 1, 0, 0,
          2, 2, 1, 0]]),
  'state': array([[[3, 2],
          [1, 3],
          [1, 3],
          [1, 2],
          [2, 3],
          [1, 3],
          [1, 2],
          [3, 2],
          [1, 2],
          [2, 1],
          [2, 3],
          [3, 2],
          [3, 1],
          [2, 3],
          [3, 2],
          [2, 1],
          [2, 1],
          [1, 3],
          [2, 1],
          [2, 1],
          [1, 3],
          [3, 2],
          [3, 1],
          [1, 3],
          [2, 3],
          [1, 2],
          [1, 3],
          [2, 1],
          [1, 3],
          [3, 1],
          [1, 2],
          [2, 3],
          [2, 1],
          [3, 1],
          [2, 1],
          [3, 2],
          [1, 2],
          [1, 2],
          [2, 3],
          [1, 3],
          [3, 2],
          [2, 3],
          [3, 1],
          [3, 1],
          [2